0. Environment + Library Setup

In [1]:
# Jupyter Notebooks sometimes do not source .bashrc env vars and path correctly, check that spark is part of path
import sys
print(sys.path)

['/home/jdelzio/data-engineering-zoomcamp/week_6_streaming/homework', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/home/jdelzio/data-engineering-zoomcamp/week_6_streaming/homework/.venv/lib/python3.11/site-packages']


In [2]:
# check that JAVA_HOME and SPARK_HOME are set
import os
print(os.environ)

environ({'USER': 'jdelzio', 'SSH_CLIENT': '73.223.183.83 50344 22', 'XDG_SESSION_TYPE': 'tty', 'SHLVL': '1', 'MOTD_SHOWN': 'pam', 'HOME': '/home/jdelzio', 'OLDPWD': '/home/jdelzio/.vscode-server', 'SSL_CERT_FILE': '/usr/lib/ssl/cert.pem', 'DBUS_SESSION_BUS_ADDRESS': 'unix:path=/run/user/1000/bus', 'LOGNAME': 'jdelzio', '_': '/home/jdelzio/data-engineering-zoomcamp/week_6_streaming/homework/.venv/bin/python', 'XDG_SESSION_CLASS': 'user', 'VSCODE_CLI_REQUIRE_TOKEN': 'ec86a67f-b4ba-44fd-a12d-ea6337092172', 'XDG_SESSION_ID': '2', 'PATH': '/home/jdelzio/data-engineering-zoomcamp/week_6_streaming/homework/.venv/bin:/home/jdelzio/.vscode-server/cli/servers/Stable-5c3e652f63e798a5ac2f31ffd0d863669328dc4c/server/bin/remote-cli:/home/jdelzio/spark/spark-3.5.1-bin-hadoop3/bin:/home/jdelzio/spark/jdk-11.0.2/bin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games', 'VSCODE_AGENT_FOLDER': '/home/jdelzio/.vscode-server', 'XDG_RUNTIME_DIR': '/run/user/1000', 'SSL_CERT_DIR': '/usr/lib/ssl/certs', 

In [3]:
# If Spark missing from path, add using sys.path.append
sys.path.append('/home/jdelzio/spark/spark-3.5.1-bin-hadoop3/python/lib/py4j-0.10.9.7-src.zip')
sys.path.append('/home/jdelzio/spark/spark-3.5.1-bin-hadoop3/python')

In [ ]:
# If JAVA_HOME and SPARK_HOME are not set, set using os.environ
os.environ["JAVA_HOME"] = "/home/jdelzio/spark/jdk-11.0.2"
os.environ["SPARK_HOME"] = "/home/jdelzio/spark/spark-3.5.1-bin-hadoop3"

In [4]:
# Load PySpark Libraries
import pyspark
from pyspark.sql import SparkSession
pyspark.__version__

'3.5.1'

1. Download Required Files

In [ ]:
!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-10.csv.gz

2. Explore Red Panda

Launch container with

``` sudo docker compose up ```

Enter container with 

``` sudo docker exec -it redpanda-1 bash ```

Check redpanda Version

``` rpk version ```

Create test-topic

``` rpk topic create test-topic ```

rpk version:
```v22.3.5 (rev 28b2443)```

rpk topic create output:

TOPIC       STATUS

test-topic  OK

3. Connect to Server

In [5]:
# Now connect to the server
import json
import time 

from kafka import KafkaProducer

def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'localhost:9092'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

In [6]:
#Q3
producer.bootstrap_connected()

True

4. Sending data to the stream

In [7]:
t0 = time.time()

topic_name = 'test-topic'

for i in range(10):
    message = {'number': i}
    producer.send(topic_name, value=message)
    print(f"Sent: {message}")
    time.sleep(0.05)

t1 = time.time()

producer.flush()

t2 = time.time()

Sent: {'number': 0}
Sent: {'number': 1}
Sent: {'number': 2}
Sent: {'number': 3}
Sent: {'number': 4}
Sent: {'number': 5}
Sent: {'number': 6}
Sent: {'number': 7}
Sent: {'number': 8}
Sent: {'number': 9}


In [8]:
print(f'sending took {(t1 - t0):.2f} seconds')

sending took 0.51 seconds


In [9]:
print(f'flushing took {(t2 - t1):.2f} seconds')

flushing took 0.00 seconds


Q4. Flushing took negligible time. Sending the data took more time.

5. Reading data with rpk

in redpanda container execute:
``` rpk topic consume test-topic ``` 

then run the send messages block above again

Now send the actual taxi data

In [10]:
import pandas as pd

In [11]:
df_green = pd.read_csv("green_tripdata_2019-10.csv.gz", compression="gzip")

/tmp/ipykernel_131386/3158814642.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_green = pd.read_csv("green_tripdata_2019-10.csv.gz", compression="gzip")


In [12]:
df_green = df_green[["lpep_pickup_datetime","lpep_dropoff_datetime","PULocationID","DOLocationID","passenger_count","trip_distance","tip_amount"]]
df_green.head(10)

,lpep_pickup_datetime,lpep_dropoff_datetime,PULocationID,DOLocationID,passenger_count,trip_distance,tip_amount
0,2019-10-01 00:26:02,2019-10-01 00:39:58,112,196,1.0,5.88,0.00
1,2019-10-01 00:18:11,2019-10-01 00:22:38,43,263,1.0,0.80,0.00
2,2019-10-01 00:09:31,2019-10-01 00:24:47,255,228,2.0,7.50,0.00
3,2019-10-01 00:37:40,2019-10-01 00:41:49,181,181,1.0,0.90,0.00
4,2019-10-01 00:08:13,2019-10-01 00:17:56,97,188,1.0,2.52,2.26
5,2019-10-01 00:35:01,2019-10-01 00:43:40,65,49,1.0,1.47,1.86
6,2019-10-01 00:28:09,2019-10-01 00:30:49,7,179,1.0,0.60,1.00
7,2019-10-01 00:28:26,2019-10-01 00:32:01,41,74,1.0,0.56,0.00
8,2019-10-01 00:14:01,2019-10-01 00:26:16,255,49,1.0,2.42,0.00
9,2019-10-01 00:03:03,2019-10-01 00:17:13,130,131,1.0,3.40,2.85


Enter redpanda-1 container and create green-trips topic:

``` rpk topic create green-trips ```

In [14]:
t0 = time.time()

for row in df_green.itertuples(index=False):
    row_dict = {col: getattr(row, col) for col in row._fields}
    producer.send("green-trips", row_dict)

t1 = time.time()
print(f'data sent in {(t1 - t0):.2f} seconds')

data sent in 47.27 seconds


6. Creating the consumer

In [15]:
# use pyspark to read the data
import pyspark
from pyspark.sql import SparkSession

pyspark_version = pyspark.__version__
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("GreenTripsConsumer") \
    .config("spark.jars.packages", kafka_jar_package) \
    .getOrCreate()

:: loading settings :: url = jar:file:/home/jdelzio/spark/spark-3.5.1-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jdelzio/.ivy2/cache
The jars for the packages stored in: /home/jdelzio/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f4611559-64ea-4353-9a02-b2020191ba8b;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.1 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 715ms :: artifacts dl 20ms
	

In [16]:
# connect to the stream
green_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "green-trips") \
    .option("startingOffsets", "earliest") \
    .load()

In [17]:
# execute function to see preview of record for each mini-batch in spark stream
def peek(mini_batch, batch_id):
    first_row = mini_batch.take(1)

    if first_row:
        print(first_row[0])

query = green_stream.writeStream.foreachBatch(peek).start()

24/04/06 23:29:26 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-8aca724e-8945-4849-8acf-20635b6ba640. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/06 23:29:26 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


24/04/06 23:29:27 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


Row(key=None, value=bytearray(b'{"lpep_pickup_datetime": "2019-10-01 00:26:02", "lpep_dropoff_datetime": "2019-10-01 00:39:58", "PULocationID": 112, "DOLocationID": 196, "passenger_count": 1.0, "trip_distance": 5.88, "tip_amount": 0.0}'), topic='green-trips', partition=0, offset=0, timestamp=datetime.datetime(2024, 4, 6, 23, 27, 54, 866000), timestampType=0)
Row(lpep_pickup_datetime='2019-10-01 00:26:02', lpep_dropoff_datetime='2019-10-01 00:39:58', PULocationID=112, DOLocationID=196, passenger_count=1.0, trip_distance=5.88, tip_amount=0.0)


In [18]:
# stop query so messages are not continually consumed
query.stop()

In [19]:
# the data is JSON, but currently in binary format. Parse it and turn it into a streaming datraframe with proper column types
from pyspark.sql import types

schema = types.StructType() \
    .add("lpep_pickup_datetime", types.StringType()) \
    .add("lpep_dropoff_datetime", types.StringType()) \
    .add("PULocationID", types.IntegerType()) \
    .add("DOLocationID", types.IntegerType()) \
    .add("passenger_count", types.DoubleType()) \
    .add("trip_distance", types.DoubleType()) \
    .add("tip_amount", types.DoubleType())

In [20]:
# apply the schema
from pyspark.sql import functions as F

green_stream = green_stream \
  .select(F.from_json(F.col("value").cast('STRING'), schema).alias("data")) \
  .select("data.*")

In [21]:
# now see what the record looks like after parsing
query = green_stream.writeStream.foreachBatch(peek).start()
# Q6: output = Row(lpep_pickup_datetime='2019-10-01 00:26:02', lpep_dropoff_datetime='2019-10-01 00:39:58', PULocationID=112, DOLocationID=196, passenger_count=1.0, trip_distance=5.88, tip_amount=0.0)

24/04/06 23:29:44 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-cccbb176-8cc0-4af0-909c-149c7ef8c3c5. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/06 23:29:44 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


24/04/06 23:29:44 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


In [22]:
query.stop()

In [23]:
# Q7: add timestamp column
green_stream = green_stream \
    .withColumn('timestamp', F.current_timestamp())

In [24]:
# Q7: Group by 5 min timestamp window and DOLocationID, order by count
popular_destinations = green_stream \
    .groupBy(F.window(F.col("timestamp"), "5 minutes"), F.col("DOLocationID")) \
    .agg(F.count("*").alias("count")) \
    .orderBy(F.col("count").desc())

In [25]:
query = popular_destinations \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .start()

query.awaitTermination()

24/04/06 23:30:01 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-a0e06889-458f-4a56-828b-923ea2d91e94. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/06 23:30:01 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/04/06 23:30:01 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-04-06 23:30:00, 2024-04-06 23:35:00}|74          |19177|
|{2024-04-06 23:30:00, 2024-04-06 23:35:00}|42          |17082|
|{2024-04-06 23:30:00, 2024-04-06 23:35:00}|41          |15108|
|{2024-04-06 23:30:00, 2024-04-06 23:35:00}|75          |13884|
|{2024-04-06 23:30:00, 2024-04-06 23:35:00}|129         |12698|
|{2024-04-06 23:30:00, 2024-04-06 23:35:00}|7           |12283|
|{2024-04-06 23:30:00, 2024-04-06 23:35:00}|166         |11844|
|{2024-04-06 23:30:00, 2024-04-06 23:35:00}|236         |8529 |
|{2024-04-06 23:30:00, 2024-04-06 23:35:00}|223         |8047 |
|{2024-04-06 23:30:00, 2024-04-06 23:35:00}|238         |7810 |
|{2024-04-06 23:30:00, 2024-04-06 23:35:00}|181         |7764 |
|{2024-

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/jdelzio/spark/spark-3.5.1-bin-hadoop3/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jdelzio/spark/spark-3.5.1-bin-hadoop3/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [26]:
query.stop()

In [27]:
spark.stop()